In [1]:
from glob import glob
import pandas as pd
import numpy as np

In [2]:
file_list = glob("data/a100/*/*output_ncu.csv")
profile_results = []
for file in file_list:
    df = pd.read_csv(file)
    used_columns = [0,6,13,14,15,16]
    unused_columns = df.columns[list(set(range(df.columns.shape[0])).difference(used_columns))]
    df = df.drop(columns=unused_columns)
    metrics = ['Duration', 'Memory Throughput', 'Compute (SM) Throughput']
    df = df[df['Metric Name'].isin(metrics) & df['Section Name'].isin(["GPU Speed Of Light Throughput"])]
    df_memory = df[0::3][['ID', 'Kernel Name', 'Metric Value']].rename(columns={'Metric Value': 'memory'})
    df_duration = df[1::3][['ID', 'Kernel Name', 'Metric Value']].rename(columns={'Metric Value': 'duration'})
    df_compute = df[2::3][['ID', 'Kernel Name', 'Metric Value']].rename(columns={'Metric Value': 'compute'})
    df = pd.merge(df_memory, df_duration, on=['ID', 'Kernel Name'])
    df = pd.merge(df, df_compute, on=['ID', 'Kernel Name'])
    df['memory'] = df['memory'].str.replace(',', '').astype(float)/100
    df['duration'] = df['duration'].str.replace(',', '').astype(float)/1e9
    df['compute'] = df['compute'].str.replace(',', '').astype(float)/100
    profile_results.append(df.groupby('Kernel Name'))

In [3]:
profile_results[0].describe()

ID               \
                                                    count         mean   
Kernel Name                                                              
ampere_sgemm_128x32_sliced1x4_tn                     12.0   670.500000   
ampere_sgemm_128x32_tn                               12.0   119.500000   
ampere_sgemm_32x128_tn                               96.0   307.875000   
ampere_sgemm_32x32_sliced1x4_tn                      24.0   385.000000   
at::native::<unnamed>::fill_reverse_indices_ker...   27.0  4635.000000   
...                                                   ...          ...   
void gemmSN_TN_kernel<float, (int)128, (int)16,...   73.0   660.095890   
void gemv2T_kernel_val<int, int, float, float, ...   27.0  4595.074074   
void gemv2T_kernel_val<int, int, float, float, ...  324.0  4485.166667   
void splitKreduce_kernel<(int)32, (int)16, int,...   12.0   649.500000   
void splitKreduce_kernel<(int)32, (int)16, int,...   12.0   671.500000   

                                                                        \
                                                            std    min   
Kernel Name                                                              
ampere_sgemm_128x32_sliced1x4_tn                      97.349884  522.0   
ampere_sgemm_128x32_tn                                61.294372   26.0   
ampere_sgemm_32x128_tn                               237.972632   15.0   
ampere_sgemm_32x32_sliced1x4_tn                      280.678371   28.0   
at::native::<unnamed>::fill_reverse_indices_ker...  2309.740895  852.0   
...                                                         ...    ...   
void gemmSN_TN_kernel<float, (int)128, (int)16,...    95.473406  499.0   
void gemv2T_kernel_val<int, int, float, float, ...  2332.589366  476.0   
void gemv2T_kernel_val<int, int, float, float, ...  2294.209778  240.0   
void splitKreduce_kernel<(int)32, (int)16, int,...    97.349884  501.0   
void splitKreduce_kernel<(int)32, (int)16, int,...    97.349884  523.0   

                                                                              \
                                                        25%     50%      75%   
Kernel Name                                                                    
ampere_sgemm_128x32_sliced1x4_tn                     596.25   670.5   744.75   
ampere_sgemm_128x32_tn                                72.75   119.5   166.25   
ampere_sgemm_32x128_tn                               114.50   220.5   476.50   
ampere_sgemm_32x32_sliced1x4_tn                      125.75   357.5   641.75   
at::native::<unnamed>::fill_reverse_indices_ker...  2743.50  4635.0  6526.50   
...                                                     ...     ...      ...   
void gemmSN_TN_kernel<float, (int)128, (int)16,...   580.00   661.0   742.00   
void gemv2T_kernel_val<int, int, float, float, ...  2716.50  4608.0  6499.50   
void gemv2T_kernel_val<int, int, float, float, ...  2523.25  4498.5  6473.75   
void splitKreduce_kernel<(int)32, (int)16, int,...   575.25   649.5   723.75   
void splitKreduce_kernel<(int)32, (int)16, int,...   597.25   671.5   745.75   

                                                           memory            \
                                                       max  count      mean   
Kernel Name                                                                   
ampere_sgemm_128x32_sliced1x4_tn                     819.0   12.0  0.243433   
ampere_sgemm_128x32_tn                               213.0   12.0  0.769817   
ampere_sgemm_32x128_tn                               810.0   96.0  0.672393   
ampere_sgemm_32x32_sliced1x4_tn                      797.0   24.0  0.537612   
at::native::<unnamed>::fill_reverse_indices_ker...  8418.0   27.0  0.051193   
...                                                    ...    ...       ...   
void gemmSN_TN_kernel<float, (int)128, (int)16,...   823.0   73.0  0.175722   
void gemv2T_kernel_val<int, int, float, float, ...  8391.0   27.0  0.8